In [20]:
import os
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-upda

In [21]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar


--2022-10-07 22:03:15--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.09s   

2022-10-07 22:03:15 (10.1 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [22]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Personal_Health").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [23]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"), sep='\t', header=True)

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     650634| R3EQSTM9PWRAL|B0091LBZSU|     578484426|Demograss Capsule...|Health & Personal...|          3|            0|          0|   N|                Y|         Three Stars|Only came with 30...| 2015-08-31|
|         US|   19827510| RBWPRK17XKIXD|B00PWW3LQ6|     456433146|Viva Labs #1 Prem...|Health & Personal

In [24]:
rows = df.count()
print(f"The number of records (rows) in the dataset is: {rows}")

The number of records (rows) in the dataset is: 5331449


In [25]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [26]:
df = df.drop("marketplace", "verified_purchase", "product_category", "review_headline", "review_body")

In [27]:
df.show()

+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|review_date|
+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|     650634| R3EQSTM9PWRAL|B0091LBZSU|     578484426|Demograss Capsule...|          3|            0|          0|   N| 2015-08-31|
|   19827510| RBWPRK17XKIXD|B00PWW3LQ6|     456433146|Viva Labs #1 Prem...|          5|            0|          0|   N| 2015-08-31|
|    1520474| RRSLOAF273XFC|B00DKEWA92|     460764511|     Whip-it Cleaner|          5|            1|          2|   N| 2015-08-31|
|   23905905|R3S8W9Q6SWIT8O|B0015R3A7M|     135102038|Optimum Nutrition...|          4|            0|          0|   N| 2015-08-31|
|   28215779|R3QQ6NSLRVBFJC|B006B8U8BG|     200666829|Wet-Stop3 Bedwett...|        

In [28]:
#fix column datatypes to match sql schema 
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType,DateType, IntegerType

df = df.withColumn("customer_id",col("customer_id").cast(IntegerType()))
df = df.withColumn("product_parent",col("product_parent").cast(IntegerType())) 
df = df.withColumn("review_date",col("review_date").cast(DateType()))
df = df.withColumn("customer_id",col("customer_id").cast(IntegerType())) 
df = df.withColumn("star_rating",col("star_rating").cast(IntegerType())) 
df = df.withColumn("helpful_votes",col("helpful_votes").cast(IntegerType())) 
df = df.withColumn("total_votes",col("total_votes").cast(IntegerType())) 


In [29]:
#check to make sure schema matches sql 
df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- review_date: date (nullable = true)



In [30]:
#drop nulls and check to make sure dataset is still sizeable 
df = df.dropna(how='any')
rows = df.count()
print(rows)

5330869


In [31]:
df.show()

+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|review_date|
+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|     650634| R3EQSTM9PWRAL|B0091LBZSU|     578484426|Demograss Capsule...|          3|            0|          0|   N| 2015-08-31|
|   19827510| RBWPRK17XKIXD|B00PWW3LQ6|     456433146|Viva Labs #1 Prem...|          5|            0|          0|   N| 2015-08-31|
|    1520474| RRSLOAF273XFC|B00DKEWA92|     460764511|     Whip-it Cleaner|          5|            1|          2|   N| 2015-08-31|
|   23905905|R3S8W9Q6SWIT8O|B0015R3A7M|     135102038|Optimum Nutrition...|          4|            0|          0|   N| 2015-08-31|
|   28215779|R3QQ6NSLRVBFJC|B006B8U8BG|     200666829|Wet-Stop3 Bedwett...|        

In [32]:
# Create dataframe to match review_id_table
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R3EQSTM9PWRAL|     650634|B0091LBZSU|     578484426| 2015-08-31|
| RBWPRK17XKIXD|   19827510|B00PWW3LQ6|     456433146| 2015-08-31|
| RRSLOAF273XFC|    1520474|B00DKEWA92|     460764511| 2015-08-31|
|R3S8W9Q6SWIT8O|   23905905|B0015R3A7M|     135102038| 2015-08-31|
|R3QQ6NSLRVBFJC|   28215779|B006B8U8BG|     200666829| 2015-08-31|
|R2XYDBMHUVJCSX|   23260912|B00PFZFD8Y|     344168617| 2015-08-31|
|R1L94ESVC657A9|    7965616|B0034792JG|     291831570| 2015-08-31|
|R3DI4B8LDWFQ3K|    3443178|B000052XIA|     493678550| 2015-08-31|
|R1G7VV4HCXUQQL|   20838781|B004GCUK8A|       3902598| 2015-08-31|
|R3TTNYN1TAMUSU|   15267517|B00K7W1QAO|     633971374| 2015-08-31|
|R3AMX2D9HMWV6Z|   30105089|B0034KYA36|     854858899| 2015-08-31|
|R3PO3K321GY8HI|    1622207|B00C7UZTT8|     264608272| 2015-08

In [33]:
# Create dataframe to match products table
products_table_df = df.select(["product_id", "product_title"])
products_table_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0091LBZSU|Demograss Capsule...|
|B00PWW3LQ6|Viva Labs #1 Prem...|
|B00DKEWA92|     Whip-it Cleaner|
|B0015R3A7M|Optimum Nutrition...|
|B006B8U8BG|Wet-Stop3 Bedwett...|
|B00PFZFD8Y|NatraCure Plantar...|
|B0034792JG|Glade Sense & Spr...|
|B000052XIA|Vagisil Anti-Itch...|
|B004GCUK8A|Glade Automatic S...|
|B00K7W1QAO|Kirkland Signatur...|
|B0034KYA36|Zyrtec Prescripti...|
|B00C7UZTT8|Pure Garcinia Cam...|
|B005F9XFN0|Now  Better Stevi...|
|B00Q8YSPTU|Now Foods Stevia ...|
|B00N34HBVG|Compression Thigh...|
|B002UN2UKS|Brownmed SEAL-TIG...|
|B003265KYK|Botanic Choice Gl...|
|B00939J5T6|Shibari SHIB-WHT ...|
|B00LJOQKRI|Eyekepper 5-pack ...|
|B002TG3QPO|Isopure Zero Carb...|
+----------+--------------------+
only showing top 20 rows



In [34]:
# Create dataframe to match customers table

customer_table_df = df.groupBy('customer_id').count()
customer_table_df = customer_table_df.withColumnRenamed('count','customer_count')
customer_table_df.show()


+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    4919528|             5|
|   51451778|             3|
|   12713799|             1|
|    8673341|             1|
|   42146698|             2|
|    1117644|             2|
|   28058398|             1|
|   14375645|             1|
|   24540309|             1|
|   39715602|             1|
|     654272|             1|
|    9015608|             5|
|   43920023|            42|
|   38209321|             1|
|   12328685|             1|
|    2866605|             2|
|   38273165|             5|
|   41066514|             1|
|   28377689|             2|
|   17018784|             2|
+-----------+--------------+
only showing top 20 rows



In [35]:
#change new customer_count column to integer type to match schema 
customer_table_df = customer_table_df.withColumn("customer_count",col("customer_count").cast(IntegerType()))
customer_table_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [36]:
# Create dataframe to match vine table
vine_table_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| R3EQSTM9PWRAL|          3|            0|          0|   N|
| RBWPRK17XKIXD|          5|            0|          0|   N|
| RRSLOAF273XFC|          5|            1|          2|   N|
|R3S8W9Q6SWIT8O|          4|            0|          0|   N|
|R3QQ6NSLRVBFJC|          4|            0|          0|   N|
|R2XYDBMHUVJCSX|          2|            0|          1|   N|
|R1L94ESVC657A9|          5|            0|          0|   N|
|R3DI4B8LDWFQ3K|          1|            3|          4|   N|
|R1G7VV4HCXUQQL|          5|            0|          0|   N|
|R3TTNYN1TAMUSU|          5|            0|          0|   N|
|R3AMX2D9HMWV6Z|          5|            0|          0|   N|
|R3PO3K321GY8HI|          1|            0|          0|   N|
| RVQ0UNYX6IEVK|          1|            0|          1|   N|
|R25E9NIGO2XQ3D|          5|            

In [39]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<amazon endpoint>:5432/postgres"
config = {"user":"postgres", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [40]:
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)
products_table_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)
customer_table_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)
vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)